In [2]:
!pip install unidecode
import pandas as pd
from unidecode import unidecode

Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'unidecode'

In [6]:
#Per Game
years = range(1998, 2025)
# 1997-98 to 2023-24
frames = {f"url{y}": f"https://www.basketball-reference.com/leagues/NBA_{y}_per_poss.html" for y in years}
pl_frames = {}
for num, y in enumerate(years, start=1):
    df = pd.read_html(frames[f"url{y}"], encoding='utf-8')[0]
    df["Season"] = f"{y-1}-{y}"
    pl_frames[f"pl{num}"] = df
NBA_Player_DF = pd.concat(pl_frames).drop_duplicates()
NBA_Player_DF.drop(NBA_Player_DF[NBA_Player_DF['Rk'] == 'Rk'].index, inplace=True)
NBA_Player_DF.reset_index(drop=True, inplace=True)
# compression_opts = dict(method='zip', archive_name='NBA_Player_Stats.csv')
# NBA_Player_DF.to_csv('NBA_Player_Stats.zip', index=False, compression=compression_opts)
NBA_Player_DF.to_csv('NBA_Player_Stats_Per_Poss.csv', index=False)
NBA_Player_DF

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Unnamed: 29,ORtg,DRtg,Season
0,1,Mahmoud Abdul-Rauf,PG,28,SAC,31,0,530,10.1,26.8,...,5.7,1.6,0.1,1.9,3.0,22.3,NaN,90,110,1997-1998
1,2,Tariq Abdul-Wahad,SG,23,SAC,59,16,959,7.8,19.4,...,2.8,1.9,0.7,3.5,4.4,20.4,NaN,91,108,1997-1998
2,3,Shareef Abdur-Rahim,SF,21,VAN,82,82,2950,11.5,23.8,...,3.8,1.6,1.3,4.5,3.6,32.3,NaN,109,112,1997-1998
3,4,Cory Alexander,PG,24,TOT,60,22,1298,7.1,16.6,...,8.7,2.9,0.5,4.6,4.1,20.2,NaN,103,106,1997-1998
4,4,Cory Alexander,PG,24,SAS,37,3,501,6.5,15.7,...,7.7,2.7,0.5,5.1,5.7,17.9,NaN,94,100,1997-1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16207,568,Thaddeus Young,PF,35,PHO,10,0,89,6.0,11.5,...,3.8,2.7,1.1,2.2,6.0,12.6,NaN,123,112,2023-2024
16208,569,Trae Young,PG,25,ATL,54,54,1942,10.7,24.9,...,14.4,1.8,0.3,5.8,2.7,34.3,NaN,117,122,2023-2024
16209,570,Omer Yurtseven,C,25,UTA,48,12,545,8.8,16.3,...,2.6,0.7,1.6,3.3,4.6,19.7,NaN,112,118,2023-2024
16210,571,Cody Zeller,C,31,NOP,43,0,320,4.0,9.5,...,6.0,1.4,0.8,2.5,6.9,11.6,NaN,119,112,2023-2024


In [13]:
# preprocessing, replacing missing values with 0, sorting
df = pd.read_csv('NBA_Player_Stats_Per_Poss.csv')
df = df[~df['Player'].str.contains('League Average', na=False)]
df.drop(columns=['Rk', 'Unnamed: 29'], inplace=True)
df.fillna(0, inplace=True)
df['Player'] = df['Player'].apply(unidecode)
traded_players = df[df['Tm'] == 'TOT'][['Player', 'Season']].drop_duplicates()
df = df.merge(traded_players, on=['Player', 'Season'], how='left', indicator=True)
df = df[(df['_merge'] == 'left_only') | (df['Tm'] == 'TOT')]
df.drop(columns=['_merge'], inplace=True)
df.sort_values(by=['Player', 'Season'], inplace=True)
df['Season'] = df['Season'].apply(lambda x: int(x.split('-')[1]) if int(x.split('-')[1]) > 50 else int(x.split('-')[1]))
position_mapping = {'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5}
df['Pos'] = df['Pos'].map(position_mapping)
# df.drop(columns=['Team'])
df.to_csv('preprocessed_stats_per_poss.csv', index=False)
print(df.head())

             Player  Pos  Age   Tm   G  GS    MP   FG   FGA    FG%  ...   TRB  \
186      A.C. Green  4.0   34  DAL  82  68  2649  4.9  10.8  0.453  ...  13.5   
724      A.C. Green  4.0   35  DAL  50  35   924  6.3  14.9  0.422  ...  13.3   
1232     A.C. Green  4.0   36  LAL  82  82  1929  4.6  10.3  0.447  ...  13.0   
1731     A.C. Green  4.0   37  MIA  82   1  1411  5.6  12.7  0.444  ...  12.2   
1110  A.J. Bramlett  5.0   23  CLE   8   0    61  3.3  17.3  0.190  ...  18.1   

      AST  STL  BLK  TOV    PF   PTS   ORtg   DRtg  Season  
186   2.5  1.6  0.5  1.4   3.2  12.1  110.0  106.0    1998  
724   1.5  1.6  0.5  1.1   4.0  14.4  102.0  104.0    1999  
1232  2.1  1.4  0.5  1.4   3.4  11.0  108.0   99.0    2000  
1731  1.5  1.2  0.3  1.8   4.6  14.3  104.0   99.0    2001  
1110  0.0  0.8  0.0  2.5  10.7   6.6   64.0  106.0    2000  

[5 rows x 31 columns]


In [4]:
#more data manipulation
df = pd.read_csv('preprocessed_stats.csv')
position_mapping = {'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5}
df['Pos'] = df['Pos'].map(position_mapping)
df.to_csv('preprocessed_stats.csv', index=False)
df.drop(columns=['Team', ])
print(df.head())

          Player   Age Team  Pos     G    GS    MP   FG  FGA    FG%  ...  ORB  \
0     A.C. Green  34.0  DAL    4  82.0  68.0  32.3  3.0  6.5  0.453  ...  2.7   
1     A.C. Green  35.0  DAL    4  50.0  35.0  18.5  2.2  5.1  0.422  ...  1.6   
2     A.C. Green  36.0  LAL    4  82.0  82.0  23.5  2.1  4.7  0.447  ...  2.0   
3     A.C. Green  37.0  MIA    4  82.0   1.0  17.2  1.8  4.0  0.444  ...  1.3   
4  A.J. Bramlett  23.0  CLE    5   8.0   0.0   7.6  0.5  2.6  0.190  ...  1.5   

   DRB  TRB  AST  STL  BLK  TOV   PF  PTS  Season  
0  5.5  8.1  1.5  1.0  0.3  0.8  1.9  7.3    1998  
1  2.9  4.6  0.5  0.6  0.2  0.4  1.4  4.9    1999  
2  4.0  5.9  1.0  0.6  0.2  0.6  1.5  5.0    2000  
3  2.5  3.8  0.5  0.4  0.1  0.5  1.5  4.5    2001  
4  1.3  2.8  0.0  0.1  0.0  0.4  1.6  1.0    2000  

[5 rows x 30 columns]


In [15]:
#Per Game
years = range(1998, 2025)
# 1997-98 to 2023-24
frames = {f"url{y}": f"https://www.basketball-reference.com/leagues/NBA_{y}_advanced.html" for y in years}
pl_frames = {}
for num, y in enumerate(years, start=1):
    df = pd.read_html(frames[f"url{y}"], encoding='utf-8')[0]
    df["Season"] = f"{y-1}-{y}"
    pl_frames[f"pl{num}"] = df
NBA_Player_DF = pd.concat(pl_frames).drop_duplicates()
NBA_Player_DF.drop(NBA_Player_DF[NBA_Player_DF['Rk'] == 'Rk'].index, inplace=True)
NBA_Player_DF.reset_index(drop=True, inplace=True)
# compression_opts = dict(method='zip', archive_name='NBA_Player_Stats.csv')
# NBA_Player_DF.to_csv('NBA_Player_Stats.zip', index=False, compression=compression_opts)
NBA_Player_DF.to_csv('NBA_Player_Stats_Advanced.csv', index=False)
NBA_Player_DF

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Season
0,1,Mahmoud Abdul-Rauf,PG,28,SAC,31,530,10.5,.405,.114,...,-0.5,0.2,-0.3,-0.025,NaN,-3.2,-1.5,-4.7,-0.4,1997-1998
1,2,Tariq Abdul-Wahad,SG,23,SAC,59,959,10.1,.456,.053,...,-0.8,0.6,-0.2,-0.008,NaN,-3.6,-1.2,-4.8,-0.7,1997-1998
2,3,Shareef Abdur-Rahim,SF,21,VAN,82,2950,21.1,.562,.038,...,6.3,0.6,6.9,.113,NaN,3.4,-1.8,1.6,2.6,1997-1998
3,4,Cory Alexander,PG,24,TOT,60,1298,15.2,.548,.440,...,1.0,1.2,2.2,.081,NaN,0.4,0.1,0.5,0.8,1997-1998
4,4,Cory Alexander,PG,24,SAS,37,501,11.1,.512,.441,...,-0.2,0.8,0.7,.065,NaN,-2.3,1.1,-1.2,0.1,1997-1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16207,568,Thaddeus Young,PF,35,PHO,10,89,15.1,.515,.048,...,0.1,0.1,0.3,.137,NaN,-1.5,0.6,-0.9,0.0,2023-2024
16208,569,Trae Young,PG,25,ATL,54,1942,20.3,.585,.465,...,4.0,0.6,4.6,.114,NaN,4.9,-2.3,2.6,2.2,2023-2024
16209,570,Omer Yurtseven,C,25,UTA,48,545,15.0,.565,.130,...,0.3,0.4,0.7,.062,NaN,-1.6,-1.5,-3.0,-0.1,2023-2024
16210,571,Cody Zeller,C,31,NOP,43,320,12.8,.483,.048,...,0.4,0.4,0.8,.124,NaN,-2.9,0.3,-2.6,0.0,2023-2024


In [20]:
# preprocessing, replacing missing values with 0, sorting
df = pd.read_csv('NBA_Player_Stats_Advanced.csv')
df = df[~df['Player'].str.contains('League Average', na=False)]
df.drop(columns=['Rk', 'Unnamed: 24'], inplace=True)
df.fillna(0, inplace=True)
df['Player'] = df['Player'].apply(unidecode)
traded_players = df[df['Tm'] == 'TOT'][['Player', 'Season']].drop_duplicates()
df = df.merge(traded_players, on=['Player', 'Season'], how='left', indicator=True)
df = df[(df['_merge'] == 'left_only') | (df['Tm'] == 'TOT')]
df.drop(columns=['_merge'], inplace=True)
df.sort_values(by=['Player', 'Season'], inplace=True)
df['Season'] = df['Season'].apply(lambda x: int(x.split('-')[1]) if int(x.split('-')[1]) > 50 else int(x.split('-')[1]))
position_mapping = {'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5}
df['Pos'] = df['Pos'].map(position_mapping)
# df.drop(columns=['Team'])
df.to_csv('preprocessed_stats_advanced.csv', index=False)
print(df.head())

             Player  Pos  Age   Tm   G    MP   PER    TS%   3PAr    FTr  ...  \
186      A.C. Green  4.0   34  DAL  82  2649  12.8  0.496  0.007  0.303  ...   
724      A.C. Green  4.0   35  DAL  50   924  12.5  0.441  0.031  0.203  ...   
1232     A.C. Green  4.0   36  LAL  82  1929  11.2  0.482  0.010  0.245  ...   
1731     A.C. Green  4.0   37  MIA  82  1411  11.2  0.492  0.019  0.343  ...   
1110  A.J. Bramlett  5.0   23  CLE   8    61  -0.4  0.190  0.000  0.000  ...   

      Unnamed: 19  OWS  DWS   WS  WS/48  OBPM  DBPM   BPM  VORP  Season  
186           0.0  2.8  2.4  5.2  0.095  -1.1  -0.3  -1.4   0.4    1998  
724           0.0  0.7  0.7  1.4  0.074  -1.1  -1.1  -2.2   0.0    1999  
1232          0.0  1.7  3.3  5.0  0.124  -1.1   0.8  -0.3   0.8    2000  
1731          0.0  1.1  2.1  3.2  0.110  -2.2   0.0  -2.2  -0.1    2001  
1110          0.0 -0.2  0.1 -0.2 -0.129  -9.2  -6.7 -15.9  -0.2    2000  

[5 rows x 28 columns]


In [12]:
df_per_poss = pd.read_csv('preprocessed_stats_per_poss.csv')
df_advanced = pd.read_csv('preprocessed_stats_advanced.csv')
merged_df = pd.merge(df_per_poss, df_advanced, on=['Player', 'Season'], suffixes=['1','1'], how='inner')
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
merged_df.to_csv('merged_preprocessed_stats.csv', index=False)

In [4]:
# def calculate_pis(player_stats):
#     offensive_score = (
#         player_stats['PER'] * 0.3 +
#         player_stats['TS%'] * 100 * 0.2 +
#         player_stats['AST%'] * 0.15 +
#         player_stats['USG%'] * 0.1 +
#         player_stats['OBPM'] * 2
#     )
#     defensive_score = (
#         player_stats['DRB%'] * 0.2 +
#         player_stats['STL%'] * 0.15 +
#         player_stats['BLK%'] * 0.15 +
#         player_stats['DBPM'] * 2
#     )
#     overall_impact = (
#         player_stats['WS/48'] * 100 * 0.3 +
#         player_stats['BPM'] * 2 +
#         player_stats['VORP'] * 5
#     )
#     pis = (offensive_score * 0.4 + defensive_score * 0.3 + overall_impact * 0.3) / 10
#     return pis
# df = pd.read_csv('merged_preprocessed_stats.csv')
# df['PIS'] = df.apply(calculate_pis, axis=1)
# df.to_csv('dataset_PIS.csv', index=False)

In [5]:
# remove data with low games played, etc
df = pd.read_csv('merged_preprocessed_stats.csv')
df.drop(columns=['Tm1'], inplace=True)
df_filtered = df[df['G1'] >= 30]
player_season_counts = df_filtered['Player'].value_counts()
players_to_keep = player_season_counts[player_season_counts > 3].index
df_filtered = df_filtered[df_filtered['Player'].isin(players_to_keep)]
df_filtered.rename(columns={'Pos1': 'Pos', 'Age1': 'Age', 'G1': 'G', 'MP1': 'MP'}, inplace=True)
df_filtered.to_csv('filtered_dataset.csv', index=False)

        Player       Age         G        GS        MP        FG       FGA  \
0   A.C. Green  1.637497  1.219307  1.139047  1.396778 -1.229128 -1.313416   
1   A.C. Green  1.872486 -0.991110  0.006269 -0.940176 -0.608822 -0.434209   
2   A.C. Green  2.107475  1.219307  1.619620  0.421354 -1.362051 -1.420636   
3   A.C. Green  2.342465  1.219307 -1.160836 -0.280410 -0.918975 -0.905978   
13  A.J. Price -0.947382 -0.576657 -1.126509 -1.020107  0.277330  0.702327   

         FG%        3P       3PA  ...     WS/48      OBPM      DBPM       BPM  \
0  -0.035635 -1.122146 -1.178438  ... -0.048972 -0.286407 -0.292974 -0.377105   
1  -0.540606 -1.122146 -1.069061  ... -0.432122 -0.286407 -0.993708 -0.669793   
2  -0.133371 -1.122146 -1.178438  ...  0.480140 -0.286407  0.670534  0.025340   
3  -0.182239 -1.122146 -1.151094  ...  0.224707 -0.738686 -0.030199 -0.669793   
13 -0.736078  1.308828  1.501316  ... -0.596330  0.124755 -0.380566 -0.047831   

        VORP  Pos_-1.4172387281179166  Pos_-